Logging the training progress
=============================

This example shows how to use the `pymia.evaluation` package to log the training progress in deep learning projects.
The Jupyter notebook can be found at `./examples/evaluation/logging_torch.ipynb`.

<div class="alert alert-info">

Note

To be able to run this example:

- Get the example data by executing `./examples/example-data/pull_example_data.py`.
- Install torch (`pip install torch`)
- You should have a basic understanding of the `pymia.data` package, see example :ref:`TODO(fabianbalsiger): title <example-data1>`.

</div>

Import the required modules.

In [1]:
import numpy as np
import pymia.data.definition as defs
import pymia.evaluation.metric as metric
import pymia.evaluation.evaluator as eval_
import pymia.evaluation.writer as writer
import torch.utils.tensorboard as tensorboard

Define the path to the dataset and the logging path.

In [2]:
hdf_file = '../example-data/example-dataset.h5'
log_path = '../example-data/log'

In this example, we show how to log predictions of segmentations of a neural network against a reference ground truth. Common metrics in medical image segmentation are the Dice coefficient, an overlap-based metric, and the Hausdorff distance, a distance-based metric. Further, we also evaluate the volume similarity, a metric that does not consider the spatial overlap.

In [3]:
metrics = [metric.DiceCoefficient(), metric.HausdorffDistance(percentile=95), metric.VolumeSimilarity()]

Now, we need to define the labels we want to evaluate. In the provided example data, we have TODO labels for different brain structures.

In [4]:
labels = {1: 'DUMMY-LABEL',  # todo(fabianbalsiger): adapt labels to example
          }

Finally, we can initialize the evaluator with the metrics and labels.

In [5]:
evaluator = eval_.SegmentationEvaluator(metrics, labels)

As we are interested in logging the training progress, we will use the commonly used TensorBoard.

In [6]:
tb = tensorboard.SummaryWriter(log_path)

Define some training parameters.

In [7]:
epochs = 20
batch_size_training = 10
batch_size_testing = 10

We can now start the training loop. After each epoch of training, we will evaluate the predictions against the references, calculate the mean and standard deviation of the metrics, and log them to the TensorBoard.

In [8]:
for epoch in epochs:
    pass

    for validation_batch in validation_dataset:
        assembled = subject_assembler.get_assembled_subject(sample[defs.KEY_SUBJECT_INDEX])
        prediction_image = pymia_conv.NumpySimpleITKImageBridge.convert(assembled, sample[defs.KEY_PROPERTIES])

        # evaluate the "prediction" against the ground truth
        evaluator.evaluate(prediction_image, label_image, sample[defs.KEY_SUBJECT])

    # calculate mean and standard deviation of each metric on the validation dataset
    results = writer.StatisticsAggregator(functions={'MEAN': np.mean, 'STD': np.std}).calculate(evaluator.results)
    # log to TensorBoard
    for result in results:
        tb.add_scalar(f'valid/{result.metric}-{result.id_}', result.value, epoch)

    # clear results such that the evaluator is ready for the next evaluation
    evaluator.clear()

TypeError: 'int' object is not iterable